# 长文档的中文摘要

## 准备

In [1]:
%%time

from llama_index.llms.openai_like import OpenAILike
from llama_index.core import Settings

llm = OpenAILike(model="qwen2-7b-6k", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True,
                 temperature=0.1,
                 request_timeout=60.0
                )

Settings.llm =llm

CPU times: user 2.92 s, sys: 530 ms, total: 3.45 s
Wall time: 4.65 s


In [2]:
%%time

# embedding
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    # model_name="dztech/bge-large-zh:v1.5",
    # model_name="bge-m3:latest",
    model_name="chatfire/bge-m3:q8_0",
    base_url="http://192.168.0.72:11435",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.embed_model = ollama_embedding

CPU times: user 543 ms, sys: 39.3 ms, total: 583 ms
Wall time: 726 ms


In [4]:
from llama_index.core import SimpleDirectoryReader

docs = SimpleDirectoryReader(
        input_files=[f"../content-short.txt", "../content.txt"]
    ).load_data()

docs[0].doc_id = "短新闻"
docs[1].doc_id = "长新闻"

docs[0].text[:100]

'扎米尔·艾哈迈德·阿旺：对接中国企业落地，巴基斯坦有三大优势\n扎米尔·艾哈迈德·阿旺\n扎米尔·艾哈迈德·阿旺\n巴基斯坦国家科技大学中国和全球化中心研究员\n\n分享到：   39\n2024-06-08 0'

In [12]:
len(docs[0].text)

3892

In [24]:
docs[1].text[:100]

'安德烈·科尔图诺夫：无论经济发展还是和平建设，亚洲都可以给欧洲上一课\n安德烈·科尔图诺夫\n安德烈·科尔图诺夫\n俄罗斯国际事务委员会学术委员会主任\n\n分享到：   1926\n2024-06-06 08:'

In [25]:
len(docs[1].text)

6290

In [3]:
%%time

from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)

CPU times: user 224 ms, sys: 23.9 ms, total: 248 ms
Wall time: 252 ms


## 基于 DocumentSummaryIndex

In [5]:
%%time

from llama_index.core import get_response_synthesizer
from llama_index.core import DocumentSummaryIndex

import nest_asyncio
nest_asyncio.apply()

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True,
)

doc_summary_index = DocumentSummaryIndex.from_documents(
    [docs[0]],
    transformations=[splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/1 [00:00<?, ?it/s]

current doc id: 短新闻


Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 171 ms, sys: 4.78 ms, total: 176 ms
Wall time: 25.4 s


In [6]:
doc_summary_index.get_document_summary("短新闻")

"The provided text discusses various aspects related to the relationship between China and Pakistan, focusing on their cooperation in economic development, security concerns, political dynamics, and historical diplomatic ties. It highlights expectations from the visit of Pakistani Prime Minister Shehbaz Sharif to China, emphasizing economic cooperation as a key focus area for Pakistan due to its current economic challenges.\n\nSome of the questions that this text can answer include:\n\n1. **Why is this visit significant for Pakistan?** - The visit signifies an opportunity for increased economic collaboration and addresses Pakistan's need for foreign investment during challenging economic times.\n2. **What are Pakistan’s expectations from economic cooperation with China?** - Expectations include leveraging Pakistan's labor market potential, utilizing affordable raw materials, and attracting Chinese investments through government-led initiatives like the Pakistani-Chinese Business Confer

In [20]:
from llama_index.core import PromptTemplate

qa_prompt_tmpl = (
    "上下文信息如下。\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "根据上下文信息而非先验知识，"
    "回答查询。\n"
    # "请同时以 {tone_name} 的语气写出答案。\n"
    "回答请使用中文。\n"
    "查询：{query_str}\n"
    "答案："
)

qa_prompt = PromptTemplate(qa_prompt_tmpl)

refine_prompt_tmpl=(
    "原始查询如下： {query_str}\n"
    "我们已提供现有答案： {existing_answer}\n"
    "我们有机会通过以下更多上下文来改进现有答案 "
    "（仅在需要时）。\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "鉴于新的上下文，改进原始答案以更好地"
    "回答查询。"
    "请使用中文回答。"
    "如果上下文无用，则返回原始答案。\n"
    "改进后的答案： "
)
refine_prompt=PromptTemplate(refine_prompt_tmpl)

summary_prompt_tmpl=(
    "Write a summary of the following. Try to use only the "
    "information provided. "
    "Try to include as many key details as possible.\n"
    "请使用中文回答。"
    "\n"
    "\n"
    "{context_str}\n"
    "\n"
    "\n"
    'SUMMARY:"""\n'
)
summary_prompt=PromptTemplate(summary_prompt_tmpl)

In [21]:
%%time

# default mode of building the index
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True,
    text_qa_template=qa_prompt,
    refine_template=refine_prompt,
    summary_template=summary_prompt,
)
doc_summary_index = DocumentSummaryIndex.from_documents(
    [docs[0]],
    transformations=[splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/1 [00:00<?, ?it/s]

current doc id: 短新闻


Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 44 ms, sys: 15.4 ms, total: 59.4 ms
Wall time: 27.1 s


In [22]:
doc_summary_index.get_document_summary("短新闻")

'扎米尔·艾哈迈德·阿旺认为，在巴基斯坦新政府成立后，总理夏巴兹·谢里夫访华的主要期待是加强与中国的经济合作。当前，巴基斯坦面临经济和政治问题，希望借助中国在海外投资、经济援助等方面的支持。中巴传统友谊深厚，但巴基斯坦需要更多外部帮助来解决经济困境。\n\n阿旺指出，巴基斯坦具有发展经济的潜力，主要体现在以下几个方面：\n1. 年轻且庞大的劳动力市场；\n2. 相对较低的劳动力成本，与中国的比较优势；\n3. 丰富的原材料资源，成本相对低廉。\n\n6月5日，深圳召开了巴基斯坦中国商务会议，旨在吸引中国企业投资。会议聚焦于高科技产业等细分领域，并强调了经济合作的前提是双边外交关系稳固。夏巴兹总理访华前已与各相关部门讨论如何深化中巴合作、加强经贸交流及鼓励中国企业赴巴发展。\n\n阿旺还提到，夏巴兹总理面临的主要挑战包括国内的贫困问题和贫富差距，以及巴基斯坦作为小国在国际政治中的脆弱性。他希望借鉴中国模式，改善巴基斯坦经济和社会状况，并与中国紧密合作以克服外部压力。\n\n中巴关系面临着来自印度、阿富汗等周边国家的挑战，尤其是印度的民族主义情绪可能对穆斯林群体及南亚地区造成影响。阿旺指出，印度与多个邻国的关系紧张，包括领土争端和外交摩擦等问题。相比之下，中国在处理周边国家关系时展现出大国风范，并与尼泊尔、缅甸等保持良好外交关系。\n\n中巴经济走廊的安全反恐问题被视为进一步巩固两国关系的关键领域。阿富汗局势的稳定对中巴商贸合作至关重要，因为阿富汗作为四战之地的历史背景影响着区域安全和经济对接的可能性。\n\n贫穷与恐怖主义之间存在紧密联系，提升巴基斯坦的国民经济水平以消除贫困根源是解决恐怖主义的关键。前任巴基斯坦总理伊姆兰·汗施政方针受到国内反对派批评，但新中国成立以来的领导人都与中国保持着高度互信的外交关系。在多党制下，每个执政党的更迭可能影响政策连续性，但中巴之间的利益和友谊不会改变。\n\n印度工业化项目强调制造业自主可控，但面临极端天气等挑战，导致民用水电短缺。印度与美国建立紧密关系后，在高科技、经济、政治和国防等领域获得支持，工业产业相对稳定。然而，印度存在民族融合问题和社会不安定因素，并且能源供应受到气候反常和工业污染的威胁。\n\n巴基斯坦在大选后的五年任期中只能解决短期问题，而中国采用民主集中制原则处理长期问题的能力被认为是一个伟大发明。本文由观察者网

In [28]:
%%time

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True,
)

doc_summary_index = DocumentSummaryIndex.from_documents(
    [docs[1]],
    transformations=[splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/1 [00:00<?, ?it/s]

current doc id: 长新闻


Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 62.8 ms, sys: 12.7 ms, total: 75.5 ms
Wall time: 27.4 s


In [29]:
doc_summary_index.get_document_summary("长新闻")

"The provided texts discuss various aspects related to global politics with a focus on Asia and its historical connections with Europe, particularly Russia. The discussions cover topics such as transformations in Harbin, China's city influenced by Russian colonialism; World War II remembrance from the perspective of Russia compared to Europe; differences in how Europeans and Asians perceive World War II, especially regarding Japan's actions during the conflict; contrasts between Europe's post-WWII recovery process and that of Asia, particularly concerning Japan; and current security challenges facing Asia.\n\nSome key questions that this text can answer include:\n1. How has Harbin transformed over time to become an important hub for economic, cultural, educational, and human exchanges between China and its northern neighbors?\n2. What role did Russia play in World War II specifically concerning the liberation of China from Japanese occupation?\n3. How do Europeans typically perceive Wo

### 长文档

In [7]:
from llama_index.core import SimpleDirectoryReader

docs = SimpleDirectoryReader(
        input_files=[f"../data/news-14k.txt"]
    ).load_data()

docs[0].doc_id = "news-14k"

docs[0].text[:100]

'欧盟对华急征电车关税，一场“汽车大战”的预言即将兑现？｜文化纵横\n\n文化纵横《文化纵横》杂志官方账号昨天 10:02\n【导读】近日，欧盟委员会公布对中国电动汽车的反补贴调查结果，称中国的纯电动汽车价值'

In [11]:
len(docs[0].text)

13869

In [8]:
%%time

splitter = SentenceSplitter(chunk_size=1024*2)

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True,
)

doc_summary_index = DocumentSummaryIndex.from_documents(
    [docs[0]],
    transformations=[splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/1 [00:00<?, ?it/s]

current doc id: news-14k


Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 112 ms, sys: 12.8 ms, total: 125 ms
Wall time: 43 s


In [9]:
doc_summary_index.get_document_summary("news-14k")

"The provided text discusses the global competition in the field of新能源汽车 (electric vehicles) with a focus on China's rapid development and its impact on international markets, particularly from the perspective of European Union actions against Chinese electric vehicle manufacturers. It highlights various aspects such as:\n\n1. **EU's Actions**: The EU has imposed tariffs on Chinese-made electric vehicles due to allegations that they benefit from unfair subsidies, which is seen as detrimental to EU producers.\n2. **China's Response**: China has criticized the EU's investigation for being unfounded and politically motivated. It argues that imposing tariffs will only temporarily protect EU markets but may lead to a decline in the global competitiveness of European automotive industries.\n3. **Global Impact**: The text suggests that electric vehicles could be as transformative as cars were in the 20th century, potentially reshaping international trade dynamics and competition across variou

In [13]:
%%time

query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", 
    streaming=True,
    similarity_top_k=5
)

CPU times: user 135 µs, sys: 20 µs, total: 155 µs
Wall time: 159 µs


In [14]:
%%time

response = query_engine.query("为什么欧盟会对中国的电动汽车征收关税？")
response.print_response_stream()

欧盟对中国的电动汽车征收关税的主要原因是为了保护本土新能源汽车产业发展、实施公平贸易政策以及确保供应链安全与国家安全。这些措施包括限制特定来源的进口，提供优惠政策支持本土产业，加强法规管理关键零部件和电池产业，并通过反补贴调查来评估潜在的不公平竞争行为。征收关税的目的在于促进本地汽车产业的发展，减少对中国低成本电动汽车的依赖，同时确保在关键技术和资源方面实现自主可控。通过设置贸易壁垒，欧盟希望为本国企业创造一个更加公平的竞争环境，同时推动绿色经济的发展和本土产业的转型。这些举措旨在支持和强化欧洲本土汽车产业，以及在市场保护、技术转移担忧、供应链安全考量、环境与可持续发展考虑、公平贸易原则以及经济利益与竞争平衡方面实现目标。CPU times: user 286 ms, sys: 30.4 ms, total: 317 ms
Wall time: 1min 15s


In [15]:
%%time

response = query_engine.query("中国如何回应欧盟的这些行动？")
response.print_response_stream()

面对欧盟可能采取的保护主义措施及对中国的投资与企业活动设置的障碍，中国可以采取以下策略来应对：

1. **增强国内产业竞争力**：通过加大研发投入、优化政策环境和提高生产效率等手段，提升新能源汽车产业的整体实力。这包括推动技术创新、产品质量和服务水平的提升。

2. **深化国际合作**：在新能源汽车领域寻求与欧盟的合作机会，特别是在标准制定和技术研发方面，以促进全球市场的互联互通和互利共赢。

3. **市场多元化战略**：加强与其他国家和地区（如美国、日本、韩国等）的新能源汽车市场合作，减少对单一市场的依赖。通过出口和投资等方式拓展国际市场。

4. **政策调整与优化**：根据国际形势的变化适时调整产业政策，包括关税政策、补贴政策以及外国企业准入规定，以适应全球竞争的新格局。

5. **技术转移与交流**：鼓励中国新能源汽车企业参与国际展会和技术合作项目，加强与其他国家在新能源领域的技术交流和转让，提升自身在全球产业链中的地位。

6. **构建绿色供应链**：推动建立更加可持续、环保的供应链体系，提高供应链透明度和责任性，以符合全球对绿色经济的需求和标准。

7. **法律与外交手段**：通过国际组织或双边谈判表达立场和诉求，寻求公平竞争环境，并可能采取法律措施保护自身利益。

此外，中国还可以：

- **多元化市场战略**：继续开拓新兴市场，如南亚、东南亚、中东等地区，利用这些市场的增长潜力和较低的人均汽车保有量作为机遇。
  
- **加强国际标准合作**：与目标国家建立检测认证联盟，简化产品进入海外市场的流程和成本。解决不同市场间的法规差异，减少风险和成本。

- **推动“新型全球化”**：
  - 在本土进行技术创新和研发，形成基于中国技术议程和市场需求的核心竞争力。
  - 积极参与南方国家的工业化进程，通过新能源汽车产能输出、基础设施建设等项目帮助当地新兴工薪阶层转变为复杂工业产品的消费者。

- **资源与供应链管理**：关注关键金属原材料供应的多元化和合作，确保供应链的安全性和稳定性。探索与矿产大国建立紧密合作关系，并可能参与“金属欧佩克”组织等实体的合作或竞争机制。

- **政策调整与谈判**：通过外交渠道与中国及其他国家进行政策对话，寻求共同利益点和解决方案。利用多边平台如世界贸易组织（WTO）参与全球规则制定和完善，争取更公平